# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Project Submission

### Libraries

In [1]:
import os

import pandas as pd

from feature_engineering import DataManager
from feature_engineering.features import TipHistory, ReorderedRatio, DynamicFeatureTest1, DynamicFeatureTest2, \
    OrderSize, ModeDepartment, PrevTippedProductsRatio, CustomerLifetime, PrevOrderTipped

### Data Preparation

In [2]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv.zip'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv.zip'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv.zip'))

data_manager = DataManager(op_prior, op_train, tip_train, tip_test, orders, products, aisles, departments)
order_amount = len(data_manager.get_orders_tip())

### Feature Engineering

In [3]:
features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'tip_history',
            'reordered_ratio', 'order_size', 'mode_dept', 'prev_tipped_products_ratio']

tip_history = TipHistory()
reordered_rate = ReorderedRatio()
order_size = OrderSize()
mode_dept = ModeDepartment()
prev_tipped_products_ratio = PrevTippedProductsRatio()
customer_lifetime = CustomerLifetime()
prev_order_tipped = PrevOrderTipped()

# Static Features
data_manager.register_feature(tip_history)
data_manager.register_feature(reordered_rate)
data_manager.register_feature(order_size)
data_manager.register_feature(customer_lifetime)
data_manager.register_feature(prev_order_tipped)

In [4]:
data_manager.compute_features()

In [5]:
data_manager.get_orders_tip().head(100)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,reordered_ratio,prev_order_tipped,order_size,customer_lifetime,tip_history
0,2539329,1,prior,1,2,8,NaN,0.0,0.000000,-1,5,0,-1.0
1,2398795,1,prior,2,3,7,15.0,0.0,0.500000,0.0,6,15,0.0
2,473747,1,prior,3,3,12,21.0,0.0,0.600000,0.0,5,36,0.0
3,2254736,1,prior,4,4,7,29.0,0.0,1.000000,0.0,5,65,0.0
4,431534,1,prior,5,4,15,28.0,0.0,0.625000,0.0,8,93,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1916106,12,prior,3,5,8,14.0,0.0,0.250000,0.0,12,44,0.0
96,1057378,12,prior,4,3,9,26.0,0.0,0.250000,0.0,20,70,0.0
97,221248,12,prior,5,1,9,30.0,0.0,0.227273,0.0,22,100,0.0
98,2618231,13,prior,1,6,12,NaN,0.0,0.000000,-1,5,0,-1.0


In [6]:
print(f"Number of orders: {len(data_manager.get_orders_tip())}")
print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

Number of orders: 3346083
Number of orders did not change: True
Number of NaN or null values in each column:
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
tip                       131209
reordered_ratio                0
prev_order_tipped              0
order_size                     0
customer_lifetime              0
tip_history                    0
dtype: int64


In [7]:
order_ids = data_manager.get_orders_tip(full=True).groupby('user_id')['order_id'].head(5)
order_amount = len(order_ids)

# Dynamic Features
dynamic_feature_test_1 = DynamicFeatureTest1()
dynamic_feature_test_2 = DynamicFeatureTest2()

data_manager.register_feature(dynamic_feature_test_1)
data_manager.register_feature(dynamic_feature_test_2)

In [8]:
data_manager.set_subset(order_ids)

In [9]:
data_manager.get_orders_tip().head(25)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,reordered_ratio,prev_order_tipped,order_size,customer_lifetime,tip_history,dynamic_feature_test_1,dynamic_feature_test_2
0,2539329,1,prior,1,2,8,NaN,0.0,0.000000,-1,5,0,-1.000000,0.2,0.2
1,2398795,1,prior,2,3,7,15.0,0.0,0.500000,0.0,6,15,0.000000,0.4,0.4
2,473747,1,prior,3,3,12,21.0,0.0,0.600000,0.0,5,36,0.000000,0.6,0.6
3,2254736,1,prior,4,4,7,29.0,0.0,1.000000,0.0,5,65,0.000000,0.8,0.8
4,431534,1,prior,5,4,15,28.0,0.0,0.625000,0.0,8,93,0.000000,1.0,1.0
5,2168274,2,prior,1,2,11,NaN,0.0,0.000000,-1,13,0,-1.000000,0.2,0.2
6,1501582,2,prior,2,5,10,10.0,0.0,0.166667,0.0,6,10,0.000000,0.4,0.4
7,1901567,2,prior,3,1,10,3.0,1.0,0.600000,0.0,5,13,0.000000,0.6,0.6
8,738281,2,prior,4,2,10,8.0,0.0,0.076923,1.0,13,21,0.333333,0.8,0.8
9,1673511,2,prior,5,3,11,8.0,1.0,0.076923,0.0,13,29,0.250000,1.0,1.0


In [10]:
print(f"Number of orders: {len(data_manager.get_orders_tip())}")
print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

Number of orders: 991222
Number of orders did not change: True

Number of NaN or null values in each column:
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
tip                        27739
reordered_ratio                0
prev_order_tipped              0
order_size                     0
customer_lifetime              0
tip_history                    0
dynamic_feature_test_1         0
dynamic_feature_test_2         0
dtype: int64


In [11]:
order_ids = data_manager.get_orders_tip(full=True).groupby('user_id')['order_id'].head(4)
order_amount = len(order_ids)

data_manager.set_subset(order_ids)
data_manager.get_orders_tip().head(25)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,reordered_ratio,prev_order_tipped,order_size,customer_lifetime,tip_history,dynamic_feature_test_1,dynamic_feature_test_2
0,2539329,1,prior,1,2,8,NaN,0.0,0.000000,-1,5,0,-1.000000,0.250000,0.250000
1,2398795,1,prior,2,3,7,15.0,0.0,0.500000,0.0,6,15,0.000000,0.500000,0.500000
2,473747,1,prior,3,3,12,21.0,0.0,0.600000,0.0,5,36,0.000000,0.750000,0.750000
3,2254736,1,prior,4,4,7,29.0,0.0,1.000000,0.0,5,65,0.000000,1.000000,1.000000
4,2168274,2,prior,1,2,11,NaN,0.0,0.000000,-1,13,0,-1.000000,0.250000,0.250000
5,1501582,2,prior,2,5,10,10.0,0.0,0.166667,0.0,6,10,0.000000,0.500000,0.500000
6,1901567,2,prior,3,1,10,3.0,1.0,0.600000,0.0,5,13,0.000000,0.750000,0.750000
7,738281,2,prior,4,2,10,8.0,0.0,0.076923,1.0,13,21,0.333333,1.000000,1.000000
8,1374495,3,prior,1,1,14,NaN,1.0,0.000000,-1,10,0,-1.000000,0.250000,0.250000
9,444309,3,prior,2,3,19,9.0,1.0,0.333333,1.0,9,9,1.000000,0.500000,0.500000


In [12]:
print(f"Number of orders: {len(data_manager.get_orders_tip())}")
print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

Number of orders: 816150
Number of orders did not change: True

Number of NaN or null values in each column:
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
tip                        15300
reordered_ratio                0
prev_order_tipped              0
order_size                     0
customer_lifetime              0
tip_history                    0
dynamic_feature_test_1         0
dynamic_feature_test_2         0
dtype: int64


### Temporary Manual Validation

In [13]:
orders_joined = data_manager.get_orders_joined()
order_size = orders_joined.groupby('order_id')['order_number'].size().reset_index().rename(
    columns={'order_number': 'order_size'})
order_size.head(25)

,order_id,order_size
0,1,8
1,2,9
2,6,3
3,10,15
4,11,5
5,13,13
6,18,28
7,20,8
8,26,8
9,30,3


In [14]:
print(f"Order size example: {order_size[order_size['order_id'] == 2168274]['order_size'].values[0]}")
orders_joined[orders_joined['order_id'] == 2168274]

Order size example: 13


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
70,2168274,2,prior,1,2,11,NaN,0.0,32792,1,0,Chipotle Beef & Pork Realstick,23,19,popcorn jerky,snacks
71,2168274,2,prior,1,2,11,NaN,0.0,47766,2,0,Organic Avocado,24,4,fresh fruits,produce
72,2168274,2,prior,1,2,11,NaN,0.0,20574,3,0,Roasted Turkey,96,20,lunch meat,deli
73,2168274,2,prior,1,2,11,NaN,0.0,12000,4,0,Baked Organic Sea Salt Crunchy Pea Snack,72,13,condiments,pantry
74,2168274,2,prior,1,2,11,NaN,0.0,48110,5,0,Thin Stackers Brown Rice Lightly Salted,78,19,crackers,snacks
75,2168274,2,prior,1,2,11,NaN,0.0,22474,6,0,Cheddar Bunnies Snack Crackers,78,19,crackers,snacks
76,2168274,2,prior,1,2,11,NaN,0.0,16589,7,0,Plantain Chips,107,19,chips pretzels,snacks
77,2168274,2,prior,1,2,11,NaN,0.0,35917,8,0,Organic Just Concord Grape Juice,98,7,juice nectars,beverages
78,2168274,2,prior,1,2,11,NaN,0.0,27344,9,0,Uncured Genoa Salami,96,20,lunch meat,deli
79,2168274,2,prior,1,2,11,NaN,0.0,30489,10,0,Original Hummus,67,20,fresh dips tapenades,deli


In [15]:
orders_joined = data_manager.get_orders_joined()
test = orders_joined.iloc[:1000]
orders_tip = data_manager.get_orders_tip().copy()


# def cumulative_union_1(user_orders):
#     cumulative_products = set()
#     for idx, order in user_orders.iterrows():
#         prev_tipped_products = cumulative_products.intersection(order['products'])
#         user_orders.at[idx, 'prev_tipped_products_ratio'] = len(prev_tipped_products) / len(order['products'])
#         if order['tip'] == 1.0:
#             cumulative_products.update(order['products'])
#     return user_orders
# 
# 
# grouped = (test.groupby(['user_id', 'order_number', 'order_id']).agg(
#     products=('product_id', lambda x: set(x)), tip=('tip', 'first'))).reset_index()
# 
# grouped = grouped.groupby('user_id').apply(cumulative_union_1, include_groups=False).reset_index(
#     drop=False).drop(columns='level_1')
# 
# final = pd.merge(orders_tip.drop('prev_tipped_products_ratio', axis=1),
#                  grouped[['user_id', 'order_number', 'prev_tipped_products_ratio']],
#                  on=['user_id', 'order_number'],
#                  how='left')
# # grouped_1.head(25)
# final.head(25)

In [16]:
# final.head(87).equals(orders_tip.head(87))

### Analysis

### Model Training & Evaluation